Financial Trading Crew with Multiple Agent System and Serper API
This project develops an automated financial trading analysis system using CrewAI to coordinate AI agents for stock market analysis, strategy development, trade execution, and risk assessment. Powered by the Serper API for real-time market data, it securely handles API keys with getpass. Four agents—Data Analyst, Trading Strategy Developer, Trade Advisor, and Risk Advisor—collaborate hierarchically under a Manager Agent to deliver insights, strategies, execution plans, and risk reports for a user-defined stock (e.g., TSLA). Key features include real-time data analysis, user-customized trading preferences, and rate-limit mitigation. The system outputs a Markdown report, ideal for traders seeking data-driven decisions. Future enhancements include implementing a custom web scraper to complement the Serper API.

In [ ]:
from crewai import Agent, Task, Crew

# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
from getpass import getpass

# Prompt user to securely input the OpenAI API key
openai_api_key = getpass("Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [3]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

/Users/junpan/.pyenv/versions/3.11.0/lib/python3.11/site-packages/pydantic/fields.py:1076: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [ ]:
# Define the Data Analyst Agent
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent uses statistical modeling and machine learning to provide crucial insights. With a knack for data, the Data Analyst Agent is the cornerstone for informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
    memory=True,  # Enable memory for context retention
    max_iter=10,  # Limit iterations to avoid excessive API calls
    cache=True  # Enable caching to reduce redundant API requests
)


In [ ]:
# Define the Trading Strategy Agent
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial markets and quantitative analysis, this agent devises and refines trading strategies. It evaluates the performance of different approaches to determine the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
    memory=True,  # Enable memory for context retention
    max_iter=10,  # Limit iterations to avoid excessive API calls
    cache=True  # Enable caching to reduce redundant API requests
)

In [ ]:
# Define the execution agent
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, and logistical details of potential trades. By evaluating these factors, it provides well-founded suggestions for when and how trades should be executed to maximize efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
    memory=True,  # Enable memory for context retention
    max_iter=10,  # Limit iterations to avoid excessive API calls
    cache=True  # Enable caching to reduce redundant API requests
)

In [ ]:
# Define the Risk Management Agent
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models and market dynamics, this agent scrutinizes the potential risks of proposed trades. It offers a detailed analysis of risk exposure and suggests safeguards to ensure that trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
    memory=True,  # Enable memory for context retention
    max_iter=10,  # Limit iterations to avoid excessive API calls
    cache=True  # Enable caching to reduce redundant API requests
)

In [10]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [11]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [13]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [14]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

In [23]:
from crewai import Agent

manager_agent = Agent(
    role="Manager",
    goal="Oversee and coordinate the trading tasks",
    backstory="Experienced financial manager with expertise in coordinating trading strategies.",
    llm="gpt-4",  # Specify the LLM to use
    verbose=True,
    allow_delegation=True,  # Manager should delegate tasks
    # Do NOT include tools parameter here
)

In [15]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

In [28]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'TSLA',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [ ]:
import requests
from getpass import getpass

api_key = getpass('Enter your Serper API key: ')
url = 'https://google.serper.dev/search'
params = {'q': 'test query', 'api_key': api_key}
response = requests.get(url, params=params)
if response.status_code == 200:
    print(response.json())
else:
    print('Error:', response.status_code)

{'searchParameters': {'q': 'test query', 'type': 'search', 'engine': 'google'}, 'organic': [{'title': 'Efficient SQL test query or validation query that will work across all ...', 'link': 'https://stackoverflow.com/questions/3668506/efficient-sql-test-query-or-validation-query-that-will-work-across-all-or-most', 'snippet': 'This is the query that will be executed just before a connection is given to you from the pool to validate that the connection to the database is still alive.', 'date': 'Sep 8, 2010', 'sitelinks': [{'title': 'Is there a command to test an SQL query without executing it ...', 'link': 'https://stackoverflow.com/questions/2433633/is-there-a-command-to-test-an-sql-query-without-executing-it-mysql-or-ansi-sq'}, {'title': 'Best way to test SQL queries [closed] - Stack Overflow', 'link': 'https://stackoverflow.com/questions/754527/best-way-to-test-sql-queries'}], 'position': 1}, {'title': 'SQL Test', 'link': 'https://sqltest.net/', 'snippet': 'Free online SQL Test tool for

In [ ]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ebbbe4b5-afbb-4d74-98a9-13f89809125d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Task: Continuously monitor and analyze market data for the selected stock (TSLA). Use statistical modeling     │
│  and machine learning to identify trends and predict market movements.                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Task: Collect and analyze market data for TSLA using statistical modeling and machine learning techniques      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  To effectively collect and analyze market data for TSLA using statistical modeling and machine learning        │
│  techniques, we will first need to gather relevant data sources such as historical stock prices, trading        │
│  volumes, financial reports, and news sentiment related to TSLA. Utilizing statistical methods like regression  │
│  analysis, time series analysis, and correlation analysis, we can identify patterns and relationships within    │
│  the data.                                                                                                      │
│                                                                                                                 │
│  Incorporating machine learning techniques such as regression algorithms, clustering, and sentiment analysis    │
│  can further enhance our ability to predict market movements and identify potential opportunities or threats    │
│  for TSLA. By training machine learning models on historical data, we can create predictive models that help    │
│  anticipate future price movements and market trends.                                                           │
│                                                                                                                 │
│  It is crucial to continuously validate our models using backtesting and cross-validation techniques to ensure  │
│  their accuracy and reliability. Regular monitoring and updating of the models will also be necessary to adapt  │
│  to changing market conditions and maintain their predictive power.                                             │
│                                                                                                                 │
│  By combining statistical modeling and machine learning approaches, we can generate valuable insights and       │
│  alerts that enable informed decision-making regarding TSLA investments and market strategies. Our goal is to   │
│  provide actionable recommendations based on data-driven analysis to capitalize on market opportunities and     │
│  mitigate risks effectively.                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  To effectively determine the user's risk tolerance and trading preferences, we need to employ a structured     │
│  approach that considers both quantitative and qualitative factors.                                             │
│                                                                                                                 │
│  1. Risk Tolerance Assessment:                                                                                  │
│     - Utilize risk tolerance questionnaires or surveys to gauge the user's comfort level with risk. These       │
│  tools can help determine if the user is a conservative, moderate, or aggressive investor.                      │
│     - Consider the user's financial goals, time horizon, and past investment experiences. These factors can     │
│  provide valuable insights into the individual's risk appetite.                                                 │
│                                                                                                                 │
│  2. Trading Preferences Analysis:                                                                               │
│     - Analyze the user's trading history to identify patterns and preferences. Look for frequency of trades,    │
│  types of securities traded, and average holding periods.                                                       │
│     - Engage in discussions with the user to understand their investment objectives, preferred trading style    │
│  (e.g., day trading, swing trading, long-term investing), and any specific industries or sectors of interest.   │
│                                                                                                                 │
│  3. Alignment with TSLA Trading Strategies:                                                                     │
│     - Once we have a comprehensive understanding of the user's risk tolerance and trading preferences, we can   │
│  tailor our trading strategies for TSLA accordingly.                                                            │
│     - For example, if the user has a high risk tolerance and prefers short-term trading, we may consider more   │
│  aggressive strategies such as momentum trading. On the other hand, if the user is risk-averse and leans        │
│  towards long-term investing, we might focus on fundamental analysis and buy-and-hold strategies for TSLA.      │
│                                                                                                                 │
│  By accurately assessing the user's risk tolerance and trading preferences, we can ensure that our trading      │
│  strategies for TSLA are aligned with their individual investment profile, ultimately maximizing the potential  │
│  for success in the market.                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Comprehensive Risk Analysis Report for TSLA:                                                                   │
│                                                                                                                 │
│  Introduction:                                                                                                  │
│  Tesla, Inc. (TSLA) is a prominent player in the electric vehicle industry with a history of market volatility  │
│  and investor interest. To support informed decision-making regarding TSLA investments, a comprehensive risk    │
│  analysis has been conducted to identify potential risks and provide mitigation recommendations.                │
│                                                                                                                 │
│  1. Market Risks:                                                                                               │
│  - Volatility Risk: TSLA's stock has exhibited high volatility in the past, influenced by factors such as       │
│  market sentiment, macroeconomic conditions, and regulatory changes. To mitigate volatility risk,               │
│  diversification of the investment portfolio and implementation of stop-loss orders can be considered.          │
│  - Competition Risk: The electric vehicle industry is highly competitive, with new entrants and technological   │
│  advancements posing a threat to TSLA's market position. Continuous monitoring of competitors and innovation    │
│  strategies can help mitigate competition risk.                                                                 │
│                                                                                                                 │
│  2. Financial Risks:                                                                                            │
│  - Liquidity Risk: TSLA's liquidity position and cash flow management are crucial considerations for            │
│  investors. Conducting regular liquidity analyses and stress testing scenarios can help mitigate liquidity      │
│  risk.                                                                                                          │
│  - Financial Performance Risk: Fluctuations in TSLA's financial performance, such as revenue growth,            │
│  profitability, and debt levels, can impact investor confidence. Thorough financial statement analysis and      │
│  trend monitoring are essential for managing financial performance risk.                                        │
│                                                                                                                 │
│  3. Operational Risks:                                                                                          │
│  - Supply Chain Risk: TSLA's reliance on suppliers and manufacturing processes exposes it to supply chain       │
│  disruptions. Developing contingency plans and diversifying suppliers can mitigate supply chain risk.           │
│  - Regulatory Risk: Changes in regulations related to electric vehicles, emissions standards, or government     │
│  incentives can affect TSLA's operations. Staying updated on regulatory developments and engaging in proactive  │
│  compliance measures can help mitigate regulatory risk.                                                         │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a7e228e1-97af-42d7-94e6-3b0109fc735e                                                                     │
│  Agent: Crew Manager                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: ebbbe4b5-afbb-4d74-98a9-13f89809125d                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Comprehensive Risk Analysis Report for TSLA:                                                     │
│                                                                                                                 │
│  Introduction:                                                                                                  │
│  Tesla, Inc. (TSLA) is a prominent player in the electric vehicle industry with a history of market volatility  │
│  and investor interest. To support informed decision-making regarding TSLA investments, a comprehensive risk    │
│  analysis has been conducted to identify potential risks and provide mitigation recommendations.                │
│                                                                                                                 │
│  1. Market Risks:                                                                                               │
│  - Volatility Risk: TSLA's stock has exhibited high volatility in the past, influenced by factors such as       │
│  market sentiment, macroeconomic conditions, and regulatory changes. To mitigate volatility risk,               │
│  diversification of the investment portfolio and implementation of stop-loss orders can be considered.          │
│  - Competition Risk: The electric vehicle industry is highly competitive, with new entrants and technological   │
│  advancements posing a threat to TSLA's market position. Continuous monitoring of competitors and innovation    │
│  strategies can help mitigate competition risk.                                                                 │
│                                                                                                                 │
│  2. Financial Risks:                                                                                            │
│  - Liquidity Risk: TSLA's liquidity position and cash flow management are crucial considerations for            │
│  investors. Conducting regular liquidity analyses and stress testing scenarios can help mitigate liquidity      │
│  risk.                                                                                                          │
│  - Financial Performance Risk: Fluctuations in TSLA's financial performance, such as revenue growth,            │
│  profitability, and debt levels, can impact investor confidence. Thorough financial statement analysis and      │
│  trend monitoring are essential for managing financial performance risk.                                        │
│                                                                                                                 │
│  3. Operational Risks:                                                                                          │
│  - Supply Chain Risk: TSLA's reliance on suppliers and manufacturing processes exposes it to supply chain       │
│  disruptions. Developing contingency plans and diversifying suppliers can mitigate supply chain risk.           │
│  - Regulatory Risk: Changes in regulations related to electric vehicles, emissions standards, or government     │
│  incentives can affect TSLA's operations. Staying updated on regulatory developments and engaging in proactive  │
│  compliance measures can help mitigate regulatory risk

In [32]:
from IPython.display import Markdown

# Extract the text content from the result object
markdown_text = result.raw  # Replace `result.raw` with the correct attribute containing the markdown text
Markdown(markdown_text)

Comprehensive Risk Analysis Report for TSLA:

Introduction:
Tesla, Inc. (TSLA) is a prominent player in the electric vehicle industry with a history of market volatility and investor interest. To support informed decision-making regarding TSLA investments, a comprehensive risk analysis has been conducted to identify potential risks and provide mitigation recommendations.

1. Market Risks:
- Volatility Risk: TSLA's stock has exhibited high volatility in the past, influenced by factors such as market sentiment, macroeconomic conditions, and regulatory changes. To mitigate volatility risk, diversification of the investment portfolio and implementation of stop-loss orders can be considered.
- Competition Risk: The electric vehicle industry is highly competitive, with new entrants and technological advancements posing a threat to TSLA's market position. Continuous monitoring of competitors and innovation strategies can help mitigate competition risk.

2. Financial Risks:
- Liquidity Risk: TSLA's liquidity position and cash flow management are crucial considerations for investors. Conducting regular liquidity analyses and stress testing scenarios can help mitigate liquidity risk.
- Financial Performance Risk: Fluctuations in TSLA's financial performance, such as revenue growth, profitability, and debt levels, can impact investor confidence. Thorough financial statement analysis and trend monitoring are essential for managing financial performance risk.

3. Operational Risks:
- Supply Chain Risk: TSLA's reliance on suppliers and manufacturing processes exposes it to supply chain disruptions. Developing contingency plans and diversifying suppliers can mitigate supply chain risk.
- Regulatory Risk: Changes in regulations related to electric vehicles, emissions standards, or government incentives can affect TSLA's operations. Staying updated on regulatory developments and engaging in proactive compliance measures can help mitigate regulatory risk.

4. Strategic Risks:
- Technological Risk: Rapid technological advancements in the electric vehicle industry require TSLA to stay innovative and adaptable. Investing in research and development and strategic partnerships can help mitigate technological risk.
- Expansion Risk: TSLA's global expansion plans introduce operational and market risks. Conducting thorough market analyses and risk assessments before entering new markets can mitigate expansion risk.

Conclusion:
The risk analysis report highlights the multifaceted risks associated with investing in TSLA and provides actionable mitigation strategies to navigate uncertainties effectively. By proactively addressing market, financial, operational, and strategic risks, investors can make informed decisions and optimize their investment outcomes in the dynamic landscape of the electric vehicle industry.